In [ ]:
import os
import numpy as np
import pandas as pd
from abc import ABC, abstractmethod
from tqdm import tqdm

In [40]:
#Load files used to train other models
x_train = pd.read_csv('/content/x_train.csv')
x_test = pd.read_csv('/content/x_test.csv')

y_train = pd.read_csv('/content/y_train.csv')
y_test = pd.read_csv('/content/y_test.csv')

In [ ]:
class MCMC(ABC):
    @abstractmethod
    def __init__():
        raise NotImplemented(f"{type(self).__name__}: init not implemented")

    @abstractmethod
    def sampler():
        raise NotImplemented(f"{type(self).__name__}: sampler not implemented")


In [ ]:
class MCMC_BNN(MCMC):
    def __init__(self, model, n_samples, n_burnin, x_data, y_data, x_test, y_test):
        self.n_samples = n_samples # number of MCMC samples
        self.n_burnin = n_burnin # number of burn-in samples
        self.x_data = x_data # (N x num_features)
        self.y_data = y_data # (N x 1)
        self.x_test = x_test # (Nt x num_features)
        self.y_test = y_test # (Nt x 1)

        # MCMC sampler hyperparameters - defines how much variation you need in changes to theta, tau
        self.step_theta = 0.025;
        self.step_eta = 0.2; # note eta is used as tau in the sampler to consider log scale.
        # Hyperpriors
        self.sigma_squared = 25
        self.nu_1 = 0
        self.nu_2 = 0

        # initisalise the linear model class
        self.model = model
        self.use_langevin_gradients = True
        self.sgd_depth = 1
        self.l_prob = 0.5 # likelihood prob
        self.theta_size = self.model.n_params # weights for each feature and a bias term

        # store output
        self.pos_theta = None
        self.pos_tau = None
        self.pos_eta = None
        self.rmse_data = None

        # functions defined above - this is poor practice, but done for readability
        # and clarity
        if self.model.data_case == 'regression':
            self.likelihood_function = self.regression_likelihood_function
            self.prior = self.regression_prior
        elif self.model.data_case == 'classification':
            self.likelihood_function = self.classification_likelihood_function
            self.prior = self.classification_prior
        else:
            raise ValueError('data_case must be regression or classification')